In [358]:
#%% MODULES

import pandas as pd
import processing
import numpy as np
import geopandas as gpd
import datetime as dt
from csv import DictWriter

In [393]:
#%% INPUT TIMESTAMPS (ts) THROUGH CSV FILES
# IMPORTANT: the csv file must have a title and be limited to one column
ts_csv_filename = '_test_timestamp2.csv'
location_ts_csv_folder = 'C:/Users/HOLGER/Downloads/SharedFiles_ProfJames/_Spline_Surface/'
location_ts_csv_file = '{}{}'.format(location_ts_csv_folder,ts_csv_filename)
ts_name_df = pd.read_csv(location_ts_csv_file)

In [394]:
#%% INPUT SUBCATCHMENTS (sc) THROUGH CSV FILE
sc_csv_filename = '_test_subcatchments.csv'
location_sc_csv_folder = 'C:/Users/HOLGER/Downloads/SharedFiles_ProfJames/_Spline_Surface/'
location_sc_csv_file = '{}{}'.format(location_sc_csv_folder,sc_csv_filename)
sc_name_df = pd.read_csv(location_sc_csv_file)

In [395]:
#%% INPUT AND OUTPUT FOLDER
input_folder = 'C:/Users/HOLGER/Downloads/SharedFiles_ProfJames/_Spline_Surface/vosc_dbf/'
output_folder = 'C:/Users/HOLGER/Downloads/SharedFiles_ProfJames/_Spline_Surface/prec_csv/'

In [396]:
#%% SET NUMBER OF ITERATIONS
# Sometimes it is necessary to put the maximum amount of iterations
# remember: the first one is inclusive, whereas the last one is exclusive
# first index: input index [n = 0] = dataframe index [n + 1 = 1]
# last index: input index [n] = dataframe index [n + 1] 
initial_index = 0
final_index = 0

ts_name_df = ts_name_df.iloc[initial_index, :] if initial_index == final_index else ts_name_df.iloc[initial_index: final_index, :]
ts_name_df = ts_name_df if type(ts_name_df) == type(pd.DataFrame()) else ts_name_df.to_frame().T
ts_name_df['vosc_dbf'] = '_' + ts_name_df.timestamp.str[3:] + '.dbf'
ts_name_df['prec_csv'] = '_' + ts_name_df.timestamp.str[3:] + '.csv'

In [397]:
ts_name_df

,timestamp,vosc_dbf,prec_csv
0,ts_202210060055,_202210060055.dbf,_202210060055.csv


In [398]:
print(ts_name_df.iloc[0,1], ts_name_df.iloc[0,2], sep='\n')

_202210060055.dbf
_202210060055.csv


In [399]:
#%% SET SUBCATCHMENT DATAFRAME
# Sometimes it is necessary to put the maximum amount of iterations
# remember: the first one is inclusive, whereas the last one is exclusive
sc_name_df['vosc_dbf'] = input_folder + 'vosc_' + sc_name_df.subcatchment.str[3:]
sc_name_df['prec_csv'] = output_folder + 'prec_' + sc_name_df.subcatchment.str[3:]

In [400]:
sc_name_df.head()

,subcatchment,area,vosc_dbf,prec_csv
0,sc_0001,1,C:/Users/HOLGER/Downloads/SharedFiles_ProfJame...,C:/Users/HOLGER/Downloads/SharedFiles_ProfJame...
1,sc_0002,2,C:/Users/HOLGER/Downloads/SharedFiles_ProfJame...,C:/Users/HOLGER/Downloads/SharedFiles_ProfJame...
2,sc_0003,3,C:/Users/HOLGER/Downloads/SharedFiles_ProfJame...,C:/Users/HOLGER/Downloads/SharedFiles_ProfJame...
3,sc_0004,4,C:/Users/HOLGER/Downloads/SharedFiles_ProfJame...,C:/Users/HOLGER/Downloads/SharedFiles_ProfJame...
4,sc_0005,5,C:/Users/HOLGER/Downloads/SharedFiles_ProfJame...,C:/Users/HOLGER/Downloads/SharedFiles_ProfJame...


In [401]:
print(sc_name_df.iloc[0,2], sc_name_df.iloc[0,3], sep='\n')

C:/Users/HOLGER/Downloads/SharedFiles_ProfJames/_Spline_Surface/vosc_dbf/vosc_0001
C:/Users/HOLGER/Downloads/SharedFiles_ProfJames/_Spline_Surface/prec_csv/prec_0001


In [402]:
#%% START TIME OF THE PROCESS
starting_time = dt.datetime.now()

In [254]:
#%% FUNCTION 2: calculating precipitation depth (pd) from attribute table sdat file
def get_prec_subca_surface_2(arg_sc_input_area, arg_sc_input_vosc_dbf_file, arg_sc_output_prec_csv_file, arg_ts_input_vosc_dbf_file, arg_ts_output_prec_csv_file):
    # sc_name_df.apply(lambda x: get_prec_subca_surface_2(x.area, x.vosc_dbf, x.prec_csv, arg_ts_input_vosc_dbf_file, arg_ts_output_prec_csv_file), axis=1)

    # CALCULATING PRECIPITATION DEPTH
    # calculating precipitation depth (pd) from attribute table sdat file
    
    input_table_vosc_dbf_file = QgsVectorLayer(arg_sc_input_vosc_dbf_file + arg_ts_input_vosc_dbf_file, '', 'ogr').getFeature(0)
    # first_feature_table = attb_table.getFeature(0)
    vol_mm_x_m2 = input_table_vosc_dbf_file[0]
    area_m2 = arg_sc_input_area # input_table_vosc_dbf_file[1]
    prec_depth_mm = vol_mm_x_m2 / area_m2 if area_m2 != 0 else 0

    # GETTING PRECIPITATION DEPTHS CSV FILES
    # Getting the precipitation depth (pd) for each subcatchment considering their volume and area
    field_names = ['PRECIPITATION_DEPTH', 'VOLUME', 'AREA']
    output_table_prec_csv_file = arg_sc_output_prec_csv_file + arg_ts_output_prec_csv_file
    first_feature_record = {'PRECIPITATION_DEPTH': prec_depth_mm, 'VOLUME': vol_mm_x_m2, 'AREA': area_m2}

    with open(output_table_prec_csv_file, 'a+', newline='') as csv_file:
        record_writer = DictWriter(csv_file, fieldnames=field_names)
        record_writer.writerow(first_feature_record)

In [255]:
#%% FUNCTION 1: Clip Raster by Mask Layer (CRBML)
def get_prec_subca_surface_1(arg_ts_input_vosc_dbf_file, arg_ts_output_prec_csv_file):
    # np.vectorize(get_prec_subca_surface_1)(ts_name_df.vosc_dbf, ts_name_df.prec_csv)

    # EXECUTE FUNCTION 2
    sc_name_df.apply(lambda x: get_prec_subca_surface_2(x.area, x.vosc_dbf, x.prec_csv, arg_ts_input_vosc_dbf_file, arg_ts_output_prec_csv_file), axis=1)
    # np.vectorize(get_vol_subca_surface_2)(sc_name_df.clsc_sdat, sc_name_df.vosc_dbf, arg_ts_input_clsc_sdat_file, arg_ts_output_vosc_dbf_file)

In [256]:
# %% EXECUTE FUNCTION 1
np.vectorize(get_prec_subca_surface_1)(ts_name_df.vosc_dbf, ts_name_df.prec_csv)

In [338]:
#%% END TIME OF THE PROCESS
ending_time = dt.datetime.now()
total_time = ending_time - starting_time
print('Done! in {}'.format(total_time))

Done! in 0:55:10.230100


In [339]:
ts_name_df.vosc_dbf

0    _cipitation_30pts_1mm_28356.dbf
Name: vosc_dbf, dtype: object

In [340]:
ts_name_df.prec_csv

0    _cipitation_30pts_1mm_28356.csv
Name: prec_csv, dtype: object

In [341]:
sc_name_df.vosc_dbf

0       C:/Users/HOLGER/Downloads/SharedFiles_ProfJame...
1       C:/Users/HOLGER/Downloads/SharedFiles_ProfJame...
2       C:/Users/HOLGER/Downloads/SharedFiles_ProfJame...
3       C:/Users/HOLGER/Downloads/SharedFiles_ProfJame...
4       C:/Users/HOLGER/Downloads/SharedFiles_ProfJame...
                              ...                        
1529    C:/Users/HOLGER/Downloads/SharedFiles_ProfJame...
1530    C:/Users/HOLGER/Downloads/SharedFiles_ProfJame...
1531    C:/Users/HOLGER/Downloads/SharedFiles_ProfJame...
1532    C:/Users/HOLGER/Downloads/SharedFiles_ProfJame...
1533    C:/Users/HOLGER/Downloads/SharedFiles_ProfJame...
Name: vosc_dbf, Length: 1534, dtype: object

In [342]:
sc_name_df.prec_csv

0       C:/Users/HOLGER/Downloads/SharedFiles_ProfJame...
1       C:/Users/HOLGER/Downloads/SharedFiles_ProfJame...
2       C:/Users/HOLGER/Downloads/SharedFiles_ProfJame...
3       C:/Users/HOLGER/Downloads/SharedFiles_ProfJame...
4       C:/Users/HOLGER/Downloads/SharedFiles_ProfJame...
                              ...                        
1529    C:/Users/HOLGER/Downloads/SharedFiles_ProfJame...
1530    C:/Users/HOLGER/Downloads/SharedFiles_ProfJame...
1531    C:/Users/HOLGER/Downloads/SharedFiles_ProfJame...
1532    C:/Users/HOLGER/Downloads/SharedFiles_ProfJame...
1533    C:/Users/HOLGER/Downloads/SharedFiles_ProfJame...
Name: prec_csv, Length: 1534, dtype: object

In [320]:
ts_name_df

,timestamp,clsc_sdat,vosc_dbf
0,Precipitation_30pts_1mm_28356,_cipitation_30pts_1mm_28356.sdat,_cipitation_30pts_1mm_28356.shp


In [321]:
sc_name_df

,subcatchment,area,clsc_sdat,vosc_dbf
0,sc_0001,1,C:/Users/HOLGER/Downloads/SharedFiles_ProfJame...,C:/Users/HOLGER/Downloads/SharedFiles_ProfJame...
1,sc_0002,2,C:/Users/HOLGER/Downloads/SharedFiles_ProfJame...,C:/Users/HOLGER/Downloads/SharedFiles_ProfJame...
2,sc_0003,3,C:/Users/HOLGER/Downloads/SharedFiles_ProfJame...,C:/Users/HOLGER/Downloads/SharedFiles_ProfJame...
3,sc_0004,4,C:/Users/HOLGER/Downloads/SharedFiles_ProfJame...,C:/Users/HOLGER/Downloads/SharedFiles_ProfJame...
4,sc_0005,5,C:/Users/HOLGER/Downloads/SharedFiles_ProfJame...,C:/Users/HOLGER/Downloads/SharedFiles_ProfJame...
...,...,...,...,...
1529,sc_1530,1530,C:/Users/HOLGER/Downloads/SharedFiles_ProfJame...,C:/Users/HOLGER/Downloads/SharedFiles_ProfJame...
1530,sc_1531,1531,C:/Users/HOLGER/Downloads/SharedFiles_ProfJame...,C:/Users/HOLGER/Downloads/SharedFiles_ProfJame...
1531,sc_1532,1532,C:/Users/HOLGER/Downloads/SharedFiles_ProfJame...,C:/Users/HOLGER/Downloads/SharedFiles_ProfJame...
1532,sc_1533,1533,C:/Users/HOLGER/Downloads/SharedFiles_ProfJame...,C:/Users/HOLGER/Downloads/SharedFiles_ProfJame...


In [258]:
ts_name_df.head()

,timestamp,clspl_sdat,clsc_sdat
0,Precipitation_30pts_1mm_28356,C:/Users/HOLGER/Downloads/SharedFiles_ProfJame...,_cipitation_30pts_1mm_28356.sdat
1,Precipitation_30pts_1mm_28357,C:/Users/HOLGER/Downloads/SharedFiles_ProfJame...,_cipitation_30pts_1mm_28357.sdat
2,Precipitation_30pts_1mm_28358,C:/Users/HOLGER/Downloads/SharedFiles_ProfJame...,_cipitation_30pts_1mm_28358.sdat
3,Precipitation_30pts_1mm_28359,C:/Users/HOLGER/Downloads/SharedFiles_ProfJame...,_cipitation_30pts_1mm_28359.sdat
4,Precipitation_30pts_1mm_28360,C:/Users/HOLGER/Downloads/SharedFiles_ProfJame...,_cipitation_30pts_1mm_28360.sdat


In [260]:
sc_name_df.head()

,subcatchment,area,clsc_sdat,mask_shp
0,sc_0001,1,C:/Users/HOLGER/Downloads/SharedFiles_ProfJame...,C:/Users/HOLGER/Downloads/SharedFiles_ProfJame...
1,sc_0002,2,C:/Users/HOLGER/Downloads/SharedFiles_ProfJame...,C:/Users/HOLGER/Downloads/SharedFiles_ProfJame...
2,sc_0003,3,C:/Users/HOLGER/Downloads/SharedFiles_ProfJame...,C:/Users/HOLGER/Downloads/SharedFiles_ProfJame...
3,sc_0004,4,C:/Users/HOLGER/Downloads/SharedFiles_ProfJame...,C:/Users/HOLGER/Downloads/SharedFiles_ProfJame...
4,sc_0005,5,C:/Users/HOLGER/Downloads/SharedFiles_ProfJame...,C:/Users/HOLGER/Downloads/SharedFiles_ProfJame...


In [218]:
ts_name_df_clspl_sdat = ts_name_df.clspl_sdat.array
ts_name_df_clspl_sdat

<PandasArray>
['C:/Users/HOLGER/Downloads/SharedFiles_ProfJames/_Spline_Surface/clspl_sdat/clspl_cipitation_30pts_1mm_28356.sdat',
 'C:/Users/HOLGER/Downloads/SharedFiles_ProfJames/_Spline_Surface/clspl_sdat/clspl_cipitation_30pts_1mm_28357.sdat',
 'C:/Users/HOLGER/Downloads/SharedFiles_ProfJames/_Spline_Surface/clspl_sdat/clspl_cipitation_30pts_1mm_28358.sdat',
 'C:/Users/HOLGER/Downloads/SharedFiles_ProfJames/_Spline_Surface/clspl_sdat/clspl_cipitation_30pts_1mm_28359.sdat',
 'C:/Users/HOLGER/Downloads/SharedFiles_ProfJames/_Spline_Surface/clspl_sdat/clspl_cipitation_30pts_1mm_28360.sdat']
Length: 5, dtype: object

In [219]:
ts_name_df_clspl_sdat[0]

'C:/Users/HOLGER/Downloads/SharedFiles_ProfJames/_Spline_Surface/clspl_sdat/clspl_cipitation_30pts_1mm_28356.sdat'

In [220]:
ts_name_df_clsc_sdat = ts_name_df.clsc_sdat.array
ts_name_df_clsc_sdat

<PandasArray>
['_cipitation_30pts_1mm_28356.sdat', '_cipitation_30pts_1mm_28357.sdat',
 '_cipitation_30pts_1mm_28358.sdat', '_cipitation_30pts_1mm_28359.sdat',
 '_cipitation_30pts_1mm_28360.sdat']
Length: 5, dtype: object

In [221]:
ts_name_df_clsc_sdat[0]

'_cipitation_30pts_1mm_28356.sdat'

In [222]:
sc_name_df_clsc_sdat = sc_name_df.clsc_sdat.array
sc_name_df_clsc_sdat

<PandasArray>
['C:/Users/HOLGER/Downloads/SharedFiles_ProfJames/_Spline_Surface/clsc_sdat/clsc_0001',
 'C:/Users/HOLGER/Downloads/SharedFiles_ProfJames/_Spline_Surface/clsc_sdat/clsc_0002',
 'C:/Users/HOLGER/Downloads/SharedFiles_ProfJames/_Spline_Surface/clsc_sdat/clsc_0003',
 'C:/Users/HOLGER/Downloads/SharedFiles_ProfJames/_Spline_Surface/clsc_sdat/clsc_0004',
 'C:/Users/HOLGER/Downloads/SharedFiles_ProfJames/_Spline_Surface/clsc_sdat/clsc_0005',
 'C:/Users/HOLGER/Downloads/SharedFiles_ProfJames/_Spline_Surface/clsc_sdat/clsc_0006',
 'C:/Users/HOLGER/Downloads/SharedFiles_ProfJames/_Spline_Surface/clsc_sdat/clsc_0007',
 'C:/Users/HOLGER/Downloads/SharedFiles_ProfJames/_Spline_Surface/clsc_sdat/clsc_0008',
 'C:/Users/HOLGER/Downloads/SharedFiles_ProfJames/_Spline_Surface/clsc_sdat/clsc_0009',
 'C:/Users/HOLGER/Downloads/SharedFiles_ProfJames/_Spline_Surface/clsc_sdat/clsc_0010',
 ...
 'C:/Users/HOLGER/Downloads/SharedFiles_ProfJames/_Spline_Surface/clsc_sdat/clsc_1525',
 'C:/Users/HO

In [223]:
sc_name_df_clsc_sdat[0]

'C:/Users/HOLGER/Downloads/SharedFiles_ProfJames/_Spline_Surface/clsc_sdat/clsc_0001'

In [224]:
sc_name_df_mask_shp = sc_name_df.mask_shp.array
sc_name_df_mask_shp

<PandasArray>
['C:/Users/HOLGER/Downloads/SharedFiles_ProfJames/_Spline_Surface/mask_sc_polygon/sc_0001.shp',
 'C:/Users/HOLGER/Downloads/SharedFiles_ProfJames/_Spline_Surface/mask_sc_polygon/sc_0002.shp',
 'C:/Users/HOLGER/Downloads/SharedFiles_ProfJames/_Spline_Surface/mask_sc_polygon/sc_0003.shp',
 'C:/Users/HOLGER/Downloads/SharedFiles_ProfJames/_Spline_Surface/mask_sc_polygon/sc_0004.shp',
 'C:/Users/HOLGER/Downloads/SharedFiles_ProfJames/_Spline_Surface/mask_sc_polygon/sc_0005.shp',
 'C:/Users/HOLGER/Downloads/SharedFiles_ProfJames/_Spline_Surface/mask_sc_polygon/sc_0006.shp',
 'C:/Users/HOLGER/Downloads/SharedFiles_ProfJames/_Spline_Surface/mask_sc_polygon/sc_0007.shp',
 'C:/Users/HOLGER/Downloads/SharedFiles_ProfJames/_Spline_Surface/mask_sc_polygon/sc_0008.shp',
 'C:/Users/HOLGER/Downloads/SharedFiles_ProfJames/_Spline_Surface/mask_sc_polygon/sc_0009.shp',
 'C:/Users/HOLGER/Downloads/SharedFiles_ProfJames/_Spline_Surface/mask_sc_polygon/sc_0010.shp',
 ...
 'C:/Users/HOLGER/Dow

In [225]:
sc_name_df_mask_shp[0]

'C:/Users/HOLGER/Downloads/SharedFiles_ProfJames/_Spline_Surface/mask_sc_polygon/sc_0001.shp'

In [226]:
sc_name_df_clsc_sdat

<PandasArray>
['C:/Users/HOLGER/Downloads/SharedFiles_ProfJames/_Spline_Surface/clsc_sdat/clsc_0001',
 'C:/Users/HOLGER/Downloads/SharedFiles_ProfJames/_Spline_Surface/clsc_sdat/clsc_0002',
 'C:/Users/HOLGER/Downloads/SharedFiles_ProfJames/_Spline_Surface/clsc_sdat/clsc_0003',
 'C:/Users/HOLGER/Downloads/SharedFiles_ProfJames/_Spline_Surface/clsc_sdat/clsc_0004',
 'C:/Users/HOLGER/Downloads/SharedFiles_ProfJames/_Spline_Surface/clsc_sdat/clsc_0005',
 'C:/Users/HOLGER/Downloads/SharedFiles_ProfJames/_Spline_Surface/clsc_sdat/clsc_0006',
 'C:/Users/HOLGER/Downloads/SharedFiles_ProfJames/_Spline_Surface/clsc_sdat/clsc_0007',
 'C:/Users/HOLGER/Downloads/SharedFiles_ProfJames/_Spline_Surface/clsc_sdat/clsc_0008',
 'C:/Users/HOLGER/Downloads/SharedFiles_ProfJames/_Spline_Surface/clsc_sdat/clsc_0009',
 'C:/Users/HOLGER/Downloads/SharedFiles_ProfJames/_Spline_Surface/clsc_sdat/clsc_0010',
 ...
 'C:/Users/HOLGER/Downloads/SharedFiles_ProfJames/_Spline_Surface/clsc_sdat/clsc_1525',
 'C:/Users/HO

In [227]:
# ts_name_df_clsc_sdat[:, np.newaxis]# + 
np.transpose(sc_name_df_clsc_sdat)

<PandasArray>
['C:/Users/HOLGER/Downloads/SharedFiles_ProfJames/_Spline_Surface/clsc_sdat/clsc_0001',
 'C:/Users/HOLGER/Downloads/SharedFiles_ProfJames/_Spline_Surface/clsc_sdat/clsc_0002',
 'C:/Users/HOLGER/Downloads/SharedFiles_ProfJames/_Spline_Surface/clsc_sdat/clsc_0003',
 'C:/Users/HOLGER/Downloads/SharedFiles_ProfJames/_Spline_Surface/clsc_sdat/clsc_0004',
 'C:/Users/HOLGER/Downloads/SharedFiles_ProfJames/_Spline_Surface/clsc_sdat/clsc_0005',
 'C:/Users/HOLGER/Downloads/SharedFiles_ProfJames/_Spline_Surface/clsc_sdat/clsc_0006',
 'C:/Users/HOLGER/Downloads/SharedFiles_ProfJames/_Spline_Surface/clsc_sdat/clsc_0007',
 'C:/Users/HOLGER/Downloads/SharedFiles_ProfJames/_Spline_Surface/clsc_sdat/clsc_0008',
 'C:/Users/HOLGER/Downloads/SharedFiles_ProfJames/_Spline_Surface/clsc_sdat/clsc_0009',
 'C:/Users/HOLGER/Downloads/SharedFiles_ProfJames/_Spline_Surface/clsc_sdat/clsc_0010',
 ...
 'C:/Users/HOLGER/Downloads/SharedFiles_ProfJames/_Spline_Surface/clsc_sdat/clsc_1525',
 'C:/Users/HO

In [228]:
ts_name_df_clsc_sdat 

<PandasArray>
['_cipitation_30pts_1mm_28356.sdat', '_cipitation_30pts_1mm_28357.sdat',
 '_cipitation_30pts_1mm_28358.sdat', '_cipitation_30pts_1mm_28359.sdat',
 '_cipitation_30pts_1mm_28360.sdat']
Length: 5, dtype: object

In [229]:
type(ts_name_df_clsc_sdat)

pandas.core.arrays.numpy_.PandasArray

In [231]:
name_df_clsc_sdat = sc_name_df_clsc_sdat + ts_name_df_clsc_sdat[:, np.newaxis]
name_df_clsc_sdat

<PandasArray>
[
['C:/Users/HOLGER/Downloads/SharedFiles_ProfJames/_Spline_Surface/clsc_sdat/clsc_0001_cipitation_30pts_1mm_28356.sdat',
 'C:/Users/HOLGER/Downloads/SharedFiles_ProfJames/_Spline_Surface/clsc_sdat/clsc_0002_cipitation_30pts_1mm_28356.sdat',
 'C:/Users/HOLGER/Downloads/SharedFiles_ProfJames/_Spline_Surface/clsc_sdat/clsc_0003_cipitation_30pts_1mm_28356.sdat',
 'C:/Users/HOLGER/Downloads/SharedFiles_ProfJames/_Spline_Surface/clsc_sdat/clsc_0004_cipitation_30pts_1mm_28356.sdat',
 'C:/Users/HOLGER/Downloads/SharedFiles_ProfJames/_Spline_Surface/clsc_sdat/clsc_0005_cipitation_30pts_1mm_28356.sdat',
 'C:/Users/HOLGER/Downloads/SharedFiles_ProfJames/_Spline_Surface/clsc_sdat/clsc_0006_cipitation_30pts_1mm_28356.sdat',
 'C:/Users/HOLGER/Downloads/SharedFiles_ProfJames/_Spline_Surface/clsc_sdat/clsc_0007_cipitation_30pts_1mm_28356.sdat',
 'C:/Users/HOLGER/Downloads/SharedFiles_ProfJames/_Spline_Surface/clsc_sdat/clsc_0008_cipitation_30pts_1mm_28356.sdat',
 'C:/Users/HOLGER/Downlo

In [234]:
name_df_clsc_sdat[0,0]

'C:/Users/HOLGER/Downloads/SharedFiles_ProfJames/_Spline_Surface/clsc_sdat/clsc_0001_cipitation_30pts_1mm_28356.sdat'

In [237]:
name_df_clsc_sdat[0,1]

'C:/Users/HOLGER/Downloads/SharedFiles_ProfJames/_Spline_Surface/clsc_sdat/clsc_0002_cipitation_30pts_1mm_28356.sdat'

In [240]:
def myfunc(a, b):
    "Return a-b if a>b, otherwise return a+b"
    if a > b:
        return a - b
    else:
        return a + b

In [243]:
vfunc = np.vectorize(myfunc)

In [245]:
vfunc([1, 2, 3, 4], 2)

array([3, 4, 1, 2])